# Setup

### additional dependencies/configuration

In [ ]:
!pip install geopandas --user -q

### modules

In [1]:
from functools import partial
import pyproj
from pyspark import SparkContext
from geopyspark.geotrellis.color import ColorMap, get_colors_from_matplotlib
from geopyspark.geotrellis.tms import TMS
from geonotebook.wrappers import TMSRasterData
from shapely.geometry import MultiPolygon, Polygon, shape
from shapely.ops import transform
import geopyspark as gps
import geopandas as gpd
import fiona

In [2]:
from overlay.data import data, mask
from overlay import analysis as an

### config spark

In [3]:
conf=gps.geopyspark_conf(appName="ClimateOverlay")
conf.set('spark.master.memory', '1200M')
sc = SparkContext(conf = conf)

### create overlay context

In [4]:
hou = an.Overlay('houston')

In [5]:
hou.set_study_area('/home/hadoop/notebooks/data/read/boundaries/houston.shp')

In [6]:
M.set_center(hou.study_area['centroid']['x'], hou.study_area['centroid']['y'], 9)

In [7]:
bounding = [hou.study_area['geom'][0]]

In [8]:
hou.layerset_from_poly(bounding, 1, 'base')

In [ ]:
M.add_layer(hou.layersets['base'].mappable_layer)

### storm surge

In [9]:
# clip raster
fulltiff = '/home/hadoop/notebooks/data/read/overlay-layers/storm-surge/US_Category5_MOM_Inundation_HighTide.tif'
maskedtiff = '/home/hadoop/notebooks/data/write/overlay-layers/storm-surge/houston_flood_category5.tif'
mask.maskRasterFromBbox(fulltiff, hou.study_area['bbox'], maskedtiff)

In [56]:
hou.layerset_from_tiff(maskedtiff, 'storm_surge')

In [57]:
M.add_layer(hou.layersets['storm_surge'].mappable_layer)

### sea level rise

In [33]:
# download and unzip sea level rise polygons
u = 'https://coast.noaa.gov/htdata/Inundation/SLR/SLRdata/TX/TX_HGX_slr_data_dist.zip'
d = "/home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/"
# data.zip_to_dir(u, d)

In [34]:
# gdb
gdb = "/home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/TX_HGX_slr_data_dist/TX_HGX_slr_final_dist.gdb"

In [35]:
# replace with unquoted filepath to gdb in order to see layers
!ogrinfo /home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/TX_HGX_slr_data_dist/TX_HGX_slr_final_dist.gdb/

Had to open data source read-only.
INFO: Open of `/home/hadoop/notebooks/data/read/overlay-layers/sea-level-rise/TX_HGX_slr_data_dist/TX_HGX_slr_final_dist.gdb/'
      using driver `OpenFileGDB' successful.
1: TX_HGX_low_0ft (Multi Polygon)
2: TX_HGX_low_1ft (Multi Polygon)
3: TX_HGX_low_2ft (Multi Polygon)
4: TX_HGX_low_3ft (Multi Polygon)
5: TX_HGX_low_4ft (Multi Polygon)
6: TX_HGX_low_5ft (Multi Polygon)
7: TX_HGX_low_6ft (Multi Polygon)
8: TX_HGX_slr_0ft (Multi Polygon)
9: TX_HGX_slr_1ft (Multi Polygon)
10: TX_HGX_slr_2ft (Multi Polygon)
11: TX_HGX_slr_3ft (Multi Polygon)
12: TX_HGX_slr_4ft (Multi Polygon)
13: TX_HGX_slr_5ft (Multi Polygon)
14: TX_HGX_slr_6ft (Multi Polygon)


In [36]:
sea_level_1 = data.read_from_gdb(gdb, 'TX_HGX_slr_1ft', MultiPolygon)

In [51]:
M.add_layer(hou.layersets['sea_level_1'].mappable_layer)

### flood

In [ ]:
# output_shp = '/home/hadoop/notebooks/data/write/overlay-layers/flood/houston_storm_surge.shp'
input_shp = '/home/hadoop/notebooks/data/read/overlay-layers/flood/Fld_Haz_ar.shp'

In [ ]:
clipped_shp = mask.clip_vector_to_boundaries(input_shp, hou)

In [ ]:
ct = data.read_from_shp(clipped_shp)

In [ ]:
hou.layerset_from_poly(ct, 1, 'flood')

In [ ]:
M.add_layer(hou.layersets['flood'].mappable_layer)

### testing euclidean distance (remove)

In [45]:
euc = gps.geotrellis.euclidean_distance(sea_level[0], source_crs='epsg:3857', zoom=13, cell_type = gps.geotrellis.constants.CellType.FLOAT64)

In [48]:
hou.layersets['euc'] = an.LayerSet()

In [49]:
hou.layersets['euc'].tiled = euc

In [50]:
hou.layersets['euc'].get_map_layer()

In [ ]:
M.add_layer(hou.layersets['euc'].mappable_layer)

### remove

In [ ]:
sl1 = data.rasterize_one_val(sea_level, 1)

In [ ]:
sl1_r = sl1.reclassify(value_map={1:1}, data_type=int, replace_nodata_with=0)

In [ ]:
sl1_reclass = an.LayerSet()

In [ ]:
sl1_reclass.build_from_poly(sea_level, 1)

In [ ]:
sl1_reclass.tiled

In [ ]:
sl1_reclass.tiled = sl1_reclass.tiled.reclassify(value_map={1:1}, data_type=int, replace_nodata_with=2)

In [ ]:
sl1_reclass.pyramid = sl1_reclass.tiled.pyramid().cache()

In [ ]:
sl1_reclass.get_map_layer()

In [ ]:
sl1_reclass.color_map = gps.ColorMap.build({1: 0xff0000ff, 2: 0x00ff00ff})

In [ ]:
sl1_reclass.tms_layer = gps.TMS.build(sl1_reclass.pyramid, sl1_reclass.color_map)

In [ ]:
sl1_reclass.mappable_layer = TMSRasterData(sl1_reclass.tms_layer)

In [ ]:
hou.layersets['sl1_reclass'] = sl1_reclass

In [ ]:
# sl1_reclass.color_map.from_colors(color_list=)

In [ ]:
c['type'] = c.geom_type.astype(str)